# **Data processing**

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
data = pd.read_csv('filtered_kz.csv')


In [43]:
train_data = pd.DataFrame()
remaining_data = data.copy()

# Add at least one interaction per user to the training set
for user in data['user_id'].unique():
    user_data = remaining_data[remaining_data['user_id'] == user]
    train_data = pd.concat([train_data, user_data.iloc[:1]])
    remaining_data = remaining_data.drop(user_data.index[:1])

# Add at least one interaction per product to the training set
for product in data['product_id'].unique():
    product_data = remaining_data[remaining_data['product_id'] == product]
    if not product_data.empty:
        train_data = pd.concat([train_data, product_data.iloc[:1]])
        remaining_data = remaining_data.drop(product_data.index[:1])
from sklearn.model_selection import train_test_split

# Split remaining data into validation and test sets
val_data, test_data = train_test_split(remaining_data, test_size=0.5, random_state=42)

# Add the remaining data to training set to balance sizes if needed
train_data = pd.concat([train_data, remaining_data])


In [44]:
print(f"Train size: {train_data.shape}, Validation size: {val_data.shape}, Test size: {test_data.shape}")
print(f"Unique users in Train: {train_data['user_id'].nunique()}, Validation: {val_data['user_id'].nunique()}, Test: {test_data['user_id'].nunique()}")
print(f"Unique products in Train: {train_data['product_id'].nunique()}, Validation: {val_data['product_id'].nunique()}, Test: {test_data['product_id'].nunique()}")

Train size: (37573, 8), Validation size: (10011, 8), Test size: (10012, 8)
Unique users in Train: 15135, Validation: 2603, Test: 2537
Unique products in Train: 2827, Validation: 1517, Test: 1495


# **Initialize Parameter**

In [45]:
import numpy as np

# Combine all data to extract unique users and products
all_data = pd.concat([train_data, val_data, test_data])

# Create mappings for user_id and product_id to indices
unique_users = all_data['user_id'].unique()
unique_products = all_data['product_id'].unique()

user_to_index = {user: idx for idx, user in enumerate(unique_users)}
product_to_index = {product: idx for idx, product in enumerate(unique_products)}

# Function to create R matrix
def create_r_matrix(data, num_products, num_users):
    R = np.zeros((num_products, num_users), dtype=int)
    for _, row in data.iterrows():
        user_idx = user_to_index[row['user_id']]
        product_idx = product_to_index[row['product_id']]
        R[product_idx, user_idx] = 1
    return R

# Number of products and users
num_products = len(unique_products)
num_users = len(unique_users)

# Create R matrices for train, validation, and test sets
R_train = create_r_matrix(train_data, num_products, num_users)
R_val = create_r_matrix(val_data, num_products, num_users)
R_test = create_r_matrix(test_data, num_products, num_users)

# Check dimensions
print("R_train shape:", R_train.shape)
print("R_val shape:", R_val.shape)
print("R_test shape:", R_test.shape)

R_train shape: (2827, 15135)
R_val shape: (2827, 15135)
R_test shape: (2827, 15135)


In [60]:
#find Y
Y_train = R_train
Y_val = R_val
Y_test = R_test

In [ ]:
#find X (optional)
#explicit

item_features = data[['product_id', 'category_code']].drop_duplicates()
# One-hot encode category codes
encoder = OneHotEncoder(sparse_output=False)
explicit_features = encoder.fit_transform(item_features[['category_code']])
# Convert to NumPy array
explicit_features = np.array(explicit_features)
# Convert explicit features to a TensorFlow constant
explicit_features = tf.constant(explicit_features, dtype=tf.float64)

#laten

num_latent_features = 10
# Randomly initialize latent features
latent_features = tf.Variable(
    tf.random.normal((explicit_features.shape[0], num_latent_features), dtype=tf.float64),
    name="latent_features"
)

# Combine explicit and latent features
X = tf.concat([explicit_features, latent_features], axis=1)

# Check the shape
print("Combined product features (X) shape:", X.shape)

Combined product features (X) shape: (2827, 20)


In [61]:
#  Useful Values
num_products, num_users = Y_train.shape
num_features = 15
# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_products,  num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)


# **Model**

In [64]:
def binary_cf_loss(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    z= tf.sigmoid(tf.matmul(X, tf.transpose(W)) + b)
    # Cast R to tf.float64 before reduce_sum to match the dtype of the numerator
    j = -tf.reduce_sum(R * (Y * tf.math.log(z) + (1 - Y) * tf.math.log(1 - z))) / tf.reduce_sum(tf.cast(R, dtype=tf.float64))
    j += (lambda_ / 2) * (tf.reduce_sum(tf.square(X)) + tf.reduce_sum(tf.square(W)))
    return j

In [65]:
iterations = 80
lambda_ = 1
for iter in range(iterations):
    with tf.GradientTape() as tape:
        cost_value = binary_cf_loss(X, W, b, Y_train, R_train, lambda_)

    grads = tape.gradient( cost_value, [X,W,b] )
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 134434.5
Training loss at iteration 20: 5255.5
Training loss at iteration 40: 866.5
Training loss at iteration 60: 111.9


In [68]:
#val loss
val_loss = binary_cf_loss(X, W, b, Y_val, R_val, lambda_)
print(f"Validation loss: {val_loss:0.4f}")
#test loss
test_loss = binary_cf_loss(X, W, b, Y_test, R_test, lambda_)
print(f"Test loss: {test_loss:0.4f}")


Validation loss: 14.5480
Test loss: 14.5478


# **Predik**

In [69]:
def predict_purchase(X, W, b):
    return tf.sigmoid(tf.matmul(X, tf.transpose(W)) + b)
# Predict purchase
Y_hat = predict_purchase(X, W, b).numpy()

def recommend_products(Y_hat, R, user_index, top_n=5):
    # Get predicted ratings for the user
    user_purchase = Y_hat[:, user_index]

    # Exclude already purchased products
    user_purchase[R[:, user_index] > 0] = -np.inf

    # Get top-N product indices
    top_products = np.argsort(user_purchase)[-top_n:][::-1]
    return top_products

# Example: Recommend top 5 products for user 0
user_index = 0
R=R_test+R_train+R_val
top_products = recommend_products(Y_hat, R, user_index, top_n=5)
print("Top recommended product indices:", top_products)

Top recommended product indices: [1409 2277 1889 1249 2435]


In [75]:
# Define input layers with the shapes of X, W, b
input_X = keras.Input(shape=X.shape[1:], dtype=X.dtype)
input_W = keras.Input(shape=W.shape[1:], dtype=W.dtype)
input_b = keras.Input(shape=b.shape[1:], dtype=b.dtype)

# Wrap the predict_purchase logic within a custom Keras Layer
class PurchasePredictionLayer(keras.layers.Layer):
    def __init__(self):
        super(PurchasePredictionLayer, self).__init__()

    def call(self, inputs):
        X, W, b = inputs
        return tf.sigmoid(tf.matmul(X, tf.transpose(W)) + b)

# Create an instance of the custom layer
purchase_prediction_layer = PurchasePredictionLayer()

# Create the model using the custom layer
model = keras.Model(inputs=[input_X, input_W, input_b],
                    outputs=purchase_prediction_layer([input_X, input_W, input_b]))

# Instead of setting weights on input layers,
# pass the initial values as input to the model during prediction
#  Example:
#  predictions = model.predict([X.numpy(), W.numpy(), b.numpy()])

# Save the model
model.save('collaborative_filtering_purchase.keras')